In [ ]:
# plan
# parse through file, based on (+) or (-) at the end of each query, and whether alignments were found, 
# add to list of false positives, false negatives, true positives, and true negatives
# keep track of length of each query in the list
# find f-1, auc, roc, sensitivity vs. specificity
# plot query length of fp, fn, tp, tn
# keep track of number of matches & query length for each positive query in a separate list, plot this


# false positive isn't perfect here, because of multiple alignments for most positives
# how can we figure out which are true positives and non-true positives, how do we filter these out?
# tracking query cover, plot this for every query alignment? make a quick histogram
# 


# questions: 
# does it make sense that a single query would match a somewhat random other protein?
# prediction probabilities??
# look at length of query vs. +/-




In [7]:
# reading in the text file
# finding some basic stats
input_f = open("refseq_unflanked_alignment.txt", "r")

t_pos = []
t_neg = []
f_pos = []
f_neg = []

preds = []
true_vals = []

lengths = []
q_num = 0

while True: 
    
    line = input_f.readline()
    
    if not line:
        break
    
    if 'NC_000001.11' in line:
                
        # parse lengths of each string
        temp = line.partition("Length:")[2]
        q_len = int(temp[1:len(temp)-1])
        lengths.append(q_len)
        
        q_num = q_num + 1
        
        # parse current query number
        # can use this as label later on
        # q_num = int(line.partition("|Query_")[2].partition(" Length")[0])
        
        # read the line 2 lines below current line
        input_f.readline()
        new_ln = input_f.readline()

        # positive sequence
        if '+' in line: 
            
            true_vals.append(1)
            

            if "No" in new_ln:
                f_pos.append(q_num)
                preds.append(0)

            elif "Sequences" in new_ln:
                t_pos.append(q_num)
                preds.append(1)

        # negative sequence
        elif '-' in line:
            
            true_vals.append(0)

            if "No" in new_ln:
                t_neg.append(q_num)
                preds.append(0)

            elif "Sequences" in new_ln:
                f_neg.append(q_num)
                preds.append(1)



In [14]:
# binarize alignments and true values

In [18]:
from tabulate import tabulate

def find_f1(t_pos, f_pos, f_neg, t_neg):
    return len(t_pos) / (len(t_pos) + 0.5*(len(f_pos) + len(f_neg)))

def find_sens(t_pos, f_pos, f_neg, t_neg):
    return len(t_pos) / (len(t_pos) + len(f_neg))

def find_spec(t_pos, f_pos, f_neg, t_neg):
    return len(t_neg) / (len(t_neg) + len(f_pos))
    
def conf_mat(t_pos, f_pos, f_neg, t_neg):
    table = [["",'actual positive', 'actual negative'], ['predicted positive', len(t_pos), len(f_pos)], ['predicted negative', len(f_neg), len(t_neg)]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
    return

def find_avg_len(list):
    sum = 0
    for item in list:
        sum = sum + lengths[item-1]
    
    return sum/len(list)
    

def plot_roc(tpr, fpr): 
    # need prediction probabilities for this
    # can also do this for ranked predictions, we can probs rank positives by something
    # arbitrary like number of alignments but idk how meaningful that is
    return

In [16]:
print("F1 score: " + str(find_f1(t_pos, f_pos, f_neg, t_neg)))
print("Sensitivity: " + str(find_sens(t_pos, f_pos, f_neg, t_neg)))
print("Specificity: " + str(find_spec(t_pos, f_pos, f_neg, t_neg)))

F1 score: 0.46226617931627606
Sensitivity: 0.5448555499239737
Specificity: 0.4704327717211761


In [11]:
conf_mat(t_pos, f_pos, f_neg, t_neg)

╒════════════════════╤═══════════════════╤═══════════════════╕
│                    │   actual positive │   actual negative │
╞════════════════════╪═══════════════════╪═══════════════════╡
│ predicted positive │              1075 │              1603 │
├────────────────────┼───────────────────┼───────────────────┤
│ predicted negative │               898 │              1424 │
╘════════════════════╧═══════════════════╧═══════════════════╛


In [22]:
print("Average Length of True Positives: " + str(find_avg_len(t_pos)))
print("Average Length of False Positives: " + str(find_avg_len(f_pos)))
print("Average Length of True Negatives: " + str(find_avg_len(t_neg)))
print("Average Length of False Negatives: " + str(find_avg_len(f_neg)))

Average Length of True Positives: 49.29953488372093
Average Length of False Positives: 48.948222083593265
Average Length of True Negatives: 53.383426966292134
Average Length of False Negatives: 47.66369710467706
